In [ ]:
! pip install mindinsight

In [8]:
import os
import argparse
from mindspore import context
import requests
import mindspore.dataset as ds
import mindspore.dataset.transforms.c_transforms as C
import mindspore.dataset.vision.c_transforms as CV
from mindspore.dataset.vision import Inter
from mindspore import dtype as mstype
import mindspore.nn as nn
from mindspore.common.initializer import Normal
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig
from mindspore.nn import Accuracy
from mindspore.train.callback import LossMonitor
from mindspore import Model
import numpy as np
from mindspore import Tensor
from mindspore import load_checkpoint, load_param_into_net
from mindspore.train.callback import SummaryCollector
import mindspore.ops as ops
from mindspore.profiler import Profiler

def download_dataset(dataset_url, path):
    filename = dataset_url.split("/")[-1]
    save_path = os.path.join(path, filename)
    if os.path.exists(save_path):
        return
    if not os.path.exists(path):
        os.makedirs(path)
    res = requests.get(dataset_url, stream=True, verify=False)
    with open(save_path, "wb") as f:
        for chunk in res.iter_content(chunk_size=512):
            if chunk:
                f.write(chunk)

def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    # Define the dataset.
    mnist_ds = ds.MnistDataset(data_path)
    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    # Define the mapping to be operated.
    resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)
    rescale_nml_op = CV.Rescale(rescale_nml, shift_nml)
    rescale_op = CV.Rescale(rescale, shift)
    hwc2chw_op = CV.HWC2CHW()
    type_cast_op = C.TypeCast(mstype.int32)

    # Use the map function to apply data operations to the dataset.
    mnist_ds = mnist_ds.map(operations=type_cast_op, input_columns="label", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=[resize_op, rescale_op, rescale_nml_op, hwc2chw_op], input_columns="image", num_parallel_workers=num_parallel_workers)


    # Perform shuffle, batch and repeat operations.
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(count=repeat_size)

    return mnist_ds


class LeNet5(nn.Cell):
    """
    Lenet network structure
    """
    def __init__(self, num_class=10, num_channel=1):
        super(LeNet5, self).__init__()
        # Define the required operation.
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.tensor_summary = ops.TensorSummary()

        self.conv_summary = ops.TensorSummary()
        # Init ImageSummary
        self.image_summary = ops.ImageSummary()

    def construct(self, x):
        # Use the defined operation to construct a forward network.
        self.image_summary("Image", x)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        self.conv_summary("Convolution", x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        self.tensor_summary("Tensor", x)
        return x

def train_net(model, epoch_size, data_path, repeat_size, ckpoint_cb, summary_collector, sink_mode):
    """Define a training method."""
    # Load the training dataset.
    ds_train = create_dataset(os.path.join(data_path, "train"), 32, repeat_size)
    model.train(epoch_size, ds_train, callbacks=[ckpoint_cb, LossMonitor(125), summary_collector], dataset_sink_mode=sink_mode)

def test_net(model, data_path):
    """Define a validation method."""
    ds_eval = create_dataset(os.path.join(data_path, "test"))
    acc = model.eval(ds_eval, dataset_sink_mode=False)
    print("{}".format(acc))

def main():
    parser = argparse.ArgumentParser(description='MindSpore LeNet Example')
    parser.add_argument('--device_target', type=str, default="CPU", choices=['Ascend', 'GPU', 'CPU'])

    args = parser.parse_known_args()[0]
    context.set_context(mode=context.GRAPH_MODE, device_target=args.device_target)

    profiler = Profiler(output_path='./profiler_data')

    train_path = "datasets/MNIST_Data/train"
    test_path = "datasets/MNIST_Data/test"

    download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-labels-idx1-ubyte",
                     train_path)
    download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-images-idx3-ubyte",
                     train_path)
    download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-labels-idx1-ubyte",
                     test_path)
    download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-images-idx3-ubyte",
                     test_path)

    net = LeNet5()
    net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
    net_opt = nn.Momentum(net.trainable_params(), learning_rate=0.01, momentum=0.9)

    config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=20)
    # Use model saving parameters.
    ckpoint = ModelCheckpoint(prefix="checkpoint_lenet", config=config_ck)

    train_epoch = 3
    mnist_path = "./datasets/MNIST_Data"
    dataset_size = 1
    model = Model(net, net_loss, net_opt, metrics={"Accuracy": Accuracy()})
    summary_collector = SummaryCollector(summary_dir='./summary_dir', collect_freq=1)
    train_net(model, train_epoch, mnist_path, dataset_size, ckpoint, summary_collector, False)

    profiler.analyse()
    test_net(model, mnist_path)

    param_dict = load_checkpoint("checkpoint_lenet-1_1875.ckpt")
    # Load parameters to the network.
    load_param_into_net(net, param_dict)

    ds_test = create_dataset(os.path.join(mnist_path, "test"), batch_size=1).create_dict_iterator()
    data = next(ds_test)

    # `images` indicates the test image, and `labels` indicates the actual classification of the test image.
    images = data["image"].asnumpy()
    labels = data["label"].asnumpy()

    # Use the model.predict function to predict the classification of the image.
    output = model.predict(Tensor(data['image']))
    predicted = np.argmax(output.asnumpy(), axis=1)

    # Output the predicted classification and the actual classification.
    print(f'Predicted: "{predicted[0]}", Actual: "{labels[0]}"')

if __name__ == "__main__":
    main()

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

epoch: 1 step: 125, loss is 2.3065526485443115
epoch: 1 step: 250, loss is 2.3161518573760986
epoch: 1 step: 375, loss is 2.297208070755005
epoch: 1 step: 500, loss is 2.301285743713379
epoch: 1 step: 625, loss is 2.326754331588745
epoch: 1 step: 750, loss is 2.286972999572754
epoch: 1 step: 875, loss is 2.3009390830993652
epoch: 1 step: 1000, loss is 2.2658631801605225
epoch: 1 step: 1125, loss is 0.5843868851661682
epoch: 1 step: 1250, loss is 0.2988366186618805
epoch: 1 step: 1375, loss is 0.19200237095355988
epoch: 1 step: 1500, loss is 0.23531019687652588
epoch: 1 step: 1625, loss is 0.3255139887332916
epoch: 1 step: 1750, loss is 0.13904906809329987
epoch: 1 step: 1875, loss is 0.21354234218597412
epoch: 2 step: 125, loss is 0.028242364525794983
epoch: 2 step: 250, loss is 0.01845042034983635
epoch: 2 step: 375, loss is 0.06367482244968414
epoch: 2 step: 500, loss is 0.12854845821857452
epoch: 2 step: 625, loss is 0.25142163038253784
epoch: 2 step: 750, loss is 0.3830878138542175